In [1]:
import os 
import pickle
import numpy as np
from os.path import join  

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.distribute import MirroredStrategy, OneDeviceStrategy

from utils import *

datasets = join(os.getcwd(), "datasets")
preprocessed_datasets = join(datasets, "preprocessed")

X_all, y_all, X_test = pickle.load(open(join(preprocessed_datasets, "cleaned_separated.pickle"), 'rb'))

strategy = OneDeviceStrategy("CPU")

In [2]:
def train_test_tuple_split(X, y, train_size, seed=None):
    def apply_tuple_split(X, split):
        train, test = [], []
        for x in X:
            train.append(x[:split])
            test.append(x[split:])
        return tuple(train), tuple(test)
    
    np.random.seed(seed)
    
    indices_ = np.arange(len(y))
    np.random.shuffle(indices_)
    
    splitter_indice = int(len(y) * train_size)
    train_indices, test_indices = np.split(indices_, [splitter_indice])
    
    X_train, X_test = apply_tuple_split(X, splitter_indice)
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test
    

X_train, X_val, y_train, y_val = train_test_tuple_split(X_all, y_all, .75, seed=42)

In [3]:
with strategy.scope():
    
    conv_len, vp_len = X_train[0].shape[-1], X_train[1].shape[-1]
    vocab_dim = int(np.unique(np.r_[X_train[1], X_test[1]]).max())+1
    
    conv_input = tf.keras.layers.Input(shape=(conv_len,))
    
    vp_input = tf.keras.layers.Input(shape=(vp_len,))
    embedded = tf.keras.layers.Embedding(vocab_dim, 128, input_length=vp_len)(vp_input)
    dense_emb = tf.keras.layers.Dense(1)(embedded)
    flatten_emb = tf.keras.layers.Flatten()(dense_emb)
    
    
    concat = tf.keras.layers.Concatenate()([conv_input, flatten_emb])
    
    output = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L1(0.01), activation='sigmoid')(concat)
    
    model = tf.keras.models.Model(inputs=[conv_input, vp_input], outputs=output)
    
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr=1e-3),
        metrics=[bcr, p1, p2, m1, m2, "accuracy"]
    )

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_bcr', factor=.5, patience=10, verbose=2),
    BCREarlyStopping(patience=30, restore_best_weights=True)
]

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 6, 128)       4480        input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 6, 1)         129         embedding[0][0]                  
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1306624)]    0                                            
______________________________________________________________________________________________

In [4]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val),
                    batch_size=120,
                    epochs=200,
                    callbacks=callbacks
                   )

Epoch 1/200
2/2 [==============================] - ETA: 2s - loss: 18.3373 - bcr: 0.4473 - p1: 0.8205 - p2: 0.0741 - m1: 39.0000 - m2: 81.0000 - accuracy: 0.316 - ETA: 0s - loss: 548.1427 - bcr: 0.4605 - p1: 0.6154 - p2: 0.3056 - m1: 38.2500 - m2: 79.5000 - accuracy: 0.40 - 6s 3s/step - loss: 724.7445 - bcr: 0.4649 - p1: 0.5470 - p2: 0.3827 - m1: 38.0000 - m2: 79.0000 - accuracy: 0.4317 - val_loss: 4276.5908 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4744


/home/romaingrx/Nextcloud/EPL/Q8/Cours/LINGI2262 - Machine Learning/LINGI2262-Projects/Assignment 5 - A Machine Learning Competition/utils.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  return bcr - bcr_delta * (1 - np.exp(-bcr_delta / sigma))


New best p value : 0.4472934603691101
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 2235.0940 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 33.0000 - m2: 87.0000 - accuracy: 0.725 - ETA: 0s - loss: 2365.2659 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 35.2500 - m2: 82.5000 - accuracy: 0.700 - 1s 467ms/step - loss: 2408.6565 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 36.0000 - m2: 81.0000 - accuracy: 0.6919 - val_loss: 2581.7910 - val_bcr: 0.5000 - val_p1: 0.0000e+00 - val_p2: 1.0000 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.4744


/home/romaingrx/Nextcloud/EPL/Q8/Cours/LINGI2262 - Machine Learning/LINGI2262-Projects/Assignment 5 - A Machine Learning Competition/utils.py:105: RuntimeWarning: invalid value encountered in double_scalars
  return bcr - bcr_delta * (1 - np.exp(-bcr_delta / sigma))


Epoch 3/200
2/2 [==============================] - ETA: 0s - loss: 1540.2231 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 38.0000 - m2: 82.0000 - accuracy: 0.683 - ETA: 0s - loss: 1297.9147 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 37.7500 - m2: 80.0000 - accuracy: 0.679 - 1s 451ms/step - loss: 1217.1452 - bcr: 0.5000 - p1: 0.0000e+00 - p2: 1.0000 - m1: 37.6667 - m2: 79.3333 - accuracy: 0.6780 - val_loss: 1109.2920 - val_bcr: 0.5000 - val_p1: 1.0000 - val_p2: 0.0000e+00 - val_m1: 41.0000 - val_m2: 37.0000 - val_accuracy: 0.5256
Epoch 4/200
2/2 [==============================] - ETA: 0s - loss: 1527.2213 - bcr: 0.5000 - p1: 1.0000 - p2: 0.0000e+00 - m1: 41.0000 - m2: 79.0000 - accuracy: 0.341 - ETA: 0s - loss: 1680.3083 - bcr: 0.5000 - p1: 1.0000 - p2: 0.0000e+00 - m1: 39.2500 - m2: 78.5000 - accuracy: 0.333 - 1s 460ms/step - loss: 1731.3373 - bcr: 0.5000 - p1: 1.0000 - p2: 0.0000e+00 - m1: 38.6667 - m2: 78.3333 - accuracy: 0.3303 - val_loss: 914.9709 - val_bcr: 0.5000 - va

In [5]:
report = Report(model, X_train, y_train, X_val, y_val).to_stdout()

------------------ Report for Functional -------------------

P score : 0.573
BCR     : 0.673
BCR hat : 0.569
